In [95]:
!pip install xlsxwriter
import numpy as np
import pandas as pd
import matplotlib as mpl
import math
import xlsxwriter     
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_rows', None)
pd.set_option("display.precision", 10)

You should consider upgrading via the 'C:\Users\trifo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [96]:
#---------------------------------------------------------------CV Stuff---------------------------------------------------------------#

############ ENTER ALL FILES, ONLY SEGMENT 5 FILES ALLOWED ############
FileList = ['Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-16 1000 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-17 900 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-18 800 mV-s.txt', 
'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-19 700 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-20 600 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-21 500 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-22 400 mV-s.txt',
'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-23 300 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-24 200 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-25 175 mV-s.txt',
'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-26 150 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-27 125 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-28 100 mV-s.txt',
'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-29 75 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-30 50 mV-s.txt', 'Fe2O3-NC-packed-Au-CME_#1 iR-comp CV-31 25 mV-s.txt']
############

############ Name of the outputted Excel file ############
ExcelFileOutputName = 'CV Data'
############

############ Fill in RHE constant
PotentialConstant = 0.676
############

############ Fill in capacitance calculation constants
Beginning = 0.4
End = 0.2
DeltaV = Beginning - End
############

############ FILL IN starting and ending x-value of integration for the peak ABOVE x-axis (from left to right of x-axis) ############
TopStartingIntegrationXValue = 0.416
TopEndingIntegrationXValue = 0.866
############

############ FILL IN starting and ending x-value of integration for the peak BELOW the x-axis (from left to right of x-axis) ############
BottomStartingIntegrationXValue = 0.416
BottomEndingIntegrationXValue = 0.866
###########

DataFrameListBeforeTreatment = []
DataFrameList = []
ScanRateList = []

for file in FileList:
    df = pd.read_csv(file)
    DataFrameList.append(df)

    for rowidx in range(df.shape[0]):
        if ''.join(x for x in df.iloc[rowidx, 0] if not x.isdigit() and x not in '.').strip() == "Scan Rate (V/s) =":
            ScanRateList.append(float(''.join(x for x in df.iloc[rowidx, 0] if x.isdigit() or x in '.')))
            break

    CycleEndValue = df.iloc[-1, 0]
    for rowidx in range(df.shape[0] - 2, -1, -1):
        if df.iloc[rowidx, 0] == CycleEndValue:
            CycleBeginning = rowidx
            break
    df.drop(df.index[0:CycleBeginning - 1], inplace = True)
    df.reset_index(drop=True, inplace = True)
    df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0])
    df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1])
    df.columns = ['Potential/V', 'Current/I']
    DataFrameListBeforeTreatment.append(df.copy(deep = True))
    df.columns = ['Potential/V vs RHE', 'Current/I']
    df.iloc[:, 0] += PotentialConstant
    df.iloc[:, 0] = df.iloc[:, 0].round(3)
    df.iloc[:, 1] = df.iloc[:, 1]
    
#---------------------------------------------------------------Capacitance Calculation Portion---------------------------------------------------------------#

ForwardCapacitanceList = []
BackwardCapacitanceList = []
TotalChargeList = []
TotalCapacitanceList = []

for index, dataframe in enumerate(DataFrameList):
    forwardcapacitance = 0
    for rowidx in range(dataframe.shape[0]):
        if dataframe.iloc[rowidx, 0] >= End:
            forwardcapacitance += (dataframe.iloc[rowidx - 2, 0] - dataframe.iloc[rowidx - 1, 0]) * (dataframe.iloc[rowidx - 2, 1] + dataframe.iloc[rowidx - 1, 1])/ (2 * ScanRateList[index])
            if dataframe.iloc[rowidx, 0] >= Beginning:
                break
    ForwardCapacitanceList.append(forwardcapacitance)

for index, dataframe in enumerate(DataFrameList):
    backwardcapacitance = 0
    for rowidx in range(dataframe.shape[0] - 1, -1, -1):
        if dataframe.iloc[rowidx, 0] >= End:
            backwardcapacitance += (dataframe.iloc[rowidx - 2, 0] - dataframe.iloc[rowidx - 1, 0]) * (dataframe.iloc[rowidx - 2, 1] + dataframe.iloc[rowidx - 1, 1])/ (2 * ScanRateList[index])
            if dataframe.iloc[rowidx, 0] >= Beginning:
                break
    BackwardCapacitanceList.append(backwardcapacitance)

for idx in range(len(FileList)):
    TotalCapacitanceList.append(abs((ForwardCapacitanceList[idx] + BackwardCapacitanceList[idx]) / (2 * DeltaV)))
    TotalChargeList.append(abs(ForwardCapacitanceList[idx] + BackwardCapacitanceList[idx]))

In [97]:
#---------------------------------------------------------------Integral AND Peak Calculation Portion---------------------------------------------------------------#

TopCurveCalculation = []
TopLineCalculation = []
TopIntegralFinalValues = []

BottomCurveCalculation = []
BottomLineCalculation = []
BottomIntegralFinalValues = []

Top_MaximumCurrentValue = []
Top_RHEValueOfMax = []

Bottom_MaximumCurrentValue = []
Bottom_RHEValueOfMax = []

#Top Integral Calculation
for idx in range(len(FileList)):
    TopCurveValue = 0
    TopLineValue = 0
    TopStartRowIndex = None
    TopEndRowIndex = None
    normaldf = DataFrameList[idx]
    modifieddf = normaldf.copy(deep = True)
    

    for rowidx in range(df.shape[0]):
        if (normaldf.iloc[rowidx, 0] == TopStartingIntegrationXValue):
            TopStartRowIndex = rowidx
            break
    for rowidx in range(df.shape[0]):
        if (normaldf.iloc[rowidx, 0] == TopEndingIntegrationXValue):
            TopEndRowIndex = rowidx
            break

    slope = (normaldf.iloc[TopEndRowIndex, 1] - normaldf.iloc[TopStartRowIndex, 1]) / (normaldf.iloc[TopEndRowIndex, 0] - normaldf.iloc[TopStartRowIndex, 0])
    yintercept = normaldf.iloc[TopEndRowIndex, 1] - (slope * normaldf.iloc[TopEndRowIndex, 0])
    for rowidx in range(TopStartRowIndex, TopEndRowIndex + 1):
        modifieddf.iloc[rowidx, 1] = (slope * normaldf.iloc[rowidx, 0]) + yintercept

    for rowidx in range(normaldf.shape[0]):
        if normaldf.iloc[rowidx, 0] >= TopStartingIntegrationXValue:
            TopCurveValue += (normaldf.iloc[rowidx - 2, 0] - normaldf.iloc[rowidx - 1, 0]) * (normaldf.iloc[rowidx - 2, 1] + normaldf.iloc[rowidx - 1, 1]) / (2 * ScanRateList[idx])
            if normaldf.iloc[rowidx, 0] >= TopEndingIntegrationXValue:
                break
    TopCurveCalculation.append(abs(TopCurveValue))

    for rowidx in range(modifieddf.shape[0]):
        if modifieddf.iloc[rowidx, 0] >= TopStartingIntegrationXValue:
            TopLineValue += (modifieddf.iloc[rowidx - 2, 0] - modifieddf.iloc[rowidx - 1, 0]) * (modifieddf.iloc[rowidx - 2, 1] + modifieddf.iloc[rowidx - 1, 1]) / (2 * ScanRateList[idx])
            if modifieddf.iloc[rowidx, 0] >= TopEndingIntegrationXValue:
                break
    TopLineCalculation.append(abs(TopLineValue))

    #Calculation for the top peak with the background subtracted
    Finding_Max_df = normaldf.loc[(normaldf['Potential/V vs RHE'] >= TopStartingIntegrationXValue) & (normaldf['Potential/V vs RHE'] <=TopEndingIntegrationXValue)]
    Value_Of_Max = Finding_Max_df['Current/I'].max()
    Index_Of_Max = Finding_Max_df['Current/I'].idxmax()
    NormalizedMaxValue = Value_Of_Max - modifieddf.iloc[Index_Of_Max, 1]
    Top_MaximumCurrentValue.append(NormalizedMaxValue)
    Top_RHEValueOfMax.append(Finding_Max_df['Potential/V vs RHE'].loc[Index_Of_Max])

#Bottom Integral Calculation
for idx in range(len(FileList)):
    BottomCurveValue = 0
    BottomLineValue = 0
    BottomStartRowIndex = None
    BottomEndRowIndex = None
    regulardf = DataFrameList[idx]
    altereddf = regulardf.copy(deep=True)

    for rowidx in range(normaldf.shape[0] - 1, -1, -1):
        if regulardf.iloc[rowidx, 0] == BottomStartingIntegrationXValue:
            BottomStartRowIndex = rowidx
            break
    
    for rowidx in range(normaldf.shape[0] - 1, -1, -1):
        if regulardf.iloc[rowidx, 0] == BottomEndingIntegrationXValue:
            BottomEndRowIndex = rowidx
            break
    
    otherslope = (regulardf.iloc[BottomEndRowIndex, 1] - regulardf.iloc[BottomStartRowIndex, 1]) / (regulardf.iloc[BottomEndRowIndex, 0] - regulardf.iloc[BottomStartRowIndex, 0])
    otheryintercept = regulardf.iloc[BottomEndRowIndex, 1] - (otherslope * regulardf.iloc[BottomEndRowIndex, 0])

    for rowidx in range(BottomStartRowIndex, BottomEndRowIndex - 1, -1):
        altereddf.iloc[rowidx, 1] = (otherslope * regulardf.iloc[rowidx, 0]) + otheryintercept

    for rowidx in range(regulardf.shape[0] - 1, -1, -1):
        if regulardf.iloc[rowidx, 0] >= BottomStartingIntegrationXValue:
            BottomCurveValue += (regulardf.iloc[rowidx - 2, 0] - regulardf.iloc[rowidx - 1, 0]) * (regulardf.iloc[rowidx - 2, 1] + regulardf.iloc[rowidx - 1, 1]) / (2 * ScanRateList[idx])
            if regulardf.iloc[rowidx, 0] >= BottomEndingIntegrationXValue:
                
                break
    BottomCurveCalculation.append(abs(BottomCurveValue))

    for rowidx in range(altereddf.shape[0] - 1, -1, -1):
        if altereddf.iloc[rowidx, 0] >= BottomStartingIntegrationXValue:
            BottomLineValue += (altereddf.iloc[rowidx - 2, 0] - altereddf.iloc[rowidx - 1, 0]) * (altereddf.iloc[rowidx - 2, 1] + altereddf.iloc[rowidx - 1, 1]) / (2 * ScanRateList[idx])
            if altereddf.iloc[rowidx, 0] >= BottomEndingIntegrationXValue:
                break
    BottomLineCalculation.append(abs(BottomLineValue))

    #Calculation for the bottom peak with the background subtracted
    Finding_Min_df = regulardf.loc[(regulardf['Potential/V vs RHE'] >= BottomStartingIntegrationXValue) & (regulardf['Potential/V vs RHE'] <=BottomEndingIntegrationXValue)]
    Value_Of_Min = Finding_Min_df['Current/I'].min()
    Index_Of_Min = Finding_Min_df['Current/I'].idxmin()
    NormalizedMinValue = Value_Of_Min - altereddf.iloc[Index_Of_Min, 1]
    Bottom_MaximumCurrentValue.append(NormalizedMinValue)
    Bottom_RHEValueOfMax.append(Finding_Min_df['Potential/V vs RHE'].loc[Index_Of_Min])

for idx in range(len(FileList)):
    TopIntegralFinalValues.append(abs(TopCurveCalculation[idx] - TopLineCalculation[idx]))
    BottomIntegralFinalValues.append(abs(BottomCurveCalculation[idx] - BottomLineCalculation[idx]))


In [98]:
#---------------------------------------------------------------Outputting to Excel File---------------------------------------------------------------#

ExcelFileOutputName = ExcelFileOutputName + '.xlsx'
SheetName = 'CV Data'
writer = pd.ExcelWriter(ExcelFileOutputName, engine='xlsxwriter')

#Inserting dataframes of first cycle of data (segments 4-5)
freecolumn = 0
freerow = 2005
for idx in range(len(FileList)):
    DataFrameListBeforeTreatment[idx].to_excel(writer, sheet_name = SheetName, index = False, startrow = 2, startcol = freecolumn)
    freecolumn += len(DataFrameListBeforeTreatment[idx].columns)
    DataFrameList[idx].to_excel(writer, sheet_name = SheetName, index = False, startrow = 2, startcol = freecolumn)
    freecolumn += len(DataFrameListBeforeTreatment[idx].columns) + 2

#Putting name of each file above dataframe in Excel file, as well as what segments the data is for
Legend = [] 
for file in FileList:
    name = file[0:len(file) - 4]
    Legend.append(name)
worksheet = writer.sheets[SheetName]
column = 0
for idx in range(len(FileList)):
    worksheet.write_string(0, column, Legend[idx])
    column += len(DataFrameListBeforeTreatment[idx].columns) + len(DataFrameListBeforeTreatment[idx].columns) + 2

column = 0
for idx in range(len(FileList)):
    worksheet.write_string(1, column, 'Segments 4-5')
    column += len(DataFrameListBeforeTreatment[idx].columns) + len(DataFrameListBeforeTreatment[idx].columns) + 2


#Inserting graph of first cycle of data (segments 4-5)
workbook = writer.book
chart = workbook.add_chart({'type': 'scatter', 'subtype': 'smooth'})
namecol = 0
categorycol = 2
valuescol = 3
for idx in range(len(FileList)):
    chart.add_series({
        'name': [SheetName, 0, namecol, 0, namecol],
        'categories': [SheetName, 3, categorycol, 3 + len(DataFrameList[idx]) - 1, categorycol],
        'values': [SheetName, 3,  valuescol, 3 + len(DataFrameList[idx]) - 1, valuescol],
    })
    namecol += 6
    categorycol += 6
    valuescol += 6
chart.set_title({'name': 'All CVs (Segments 4-5)'})
chart.set_x_axis({'min': 0, 'max': 1, 'name': 'Potential, V vs RHE'})
chart.set_y_axis({'name': 'Current, A'})
worksheet.insert_chart(0, freecolumn, chart)

#Inserting calculation table of first cycle of data (segments 4-5) 
TableHeader = []
for idx in range(len(FileList)):
    ForwardCapacitanceList[idx] = abs(ForwardCapacitanceList[idx])
    BackwardCapacitanceList[idx] = abs(BackwardCapacitanceList[idx])
    TableHeader.append(Legend[idx][34:])
TableList = [ScanRateList, ForwardCapacitanceList, BackwardCapacitanceList, TotalChargeList, TotalCapacitanceList, TopIntegralFinalValues, BottomIntegralFinalValues, Top_RHEValueOfMax, Top_MaximumCurrentValue, Bottom_RHEValueOfMax, Bottom_MaximumCurrentValue]
dftable = pd.DataFrame(TableList).T
dftable.columns = ['Scan Rate (V/s)', 'Double Layer Charge Pos (mA)', 'Double Layer Charge Neg (mA)', 'Total Charge (mA)', 'Capacitance (mF)', 'Top Charge (mCs)', 'Bottom Charge (mCs)', 'Epa', 'Ipa', 'Eca', 'Ica']
dftable.index = TableHeader
dftable.to_excel(writer, sheet_name = SheetName, startrow = 32, startcol = freecolumn)
worksheet.write_string(31, freecolumn, 'Segments 4-5 Table')

0

0

0

0

0

...

0

0

0

0

0

Below is just repeating the process above, except for segments 2-3. 

In [99]:
DataFrameListBeforeTreatment = []
DataFrameList = []
ScanRateList = []

for file in FileList:
    df = pd.read_csv(file)
    DataFrameList.append(df)

    for rowidx in range(df.shape[0]):
        if ''.join(x for x in df.iloc[rowidx, 0] if not x.isdigit() and x not in '.').strip() == "Scan Rate (V/s) =":
            ScanRateList.append(float(''.join(x for x in df.iloc[rowidx, 0] if x.isdigit() or x in '.')))
            break

    CycleEndValue = df.iloc[-1, 0]
    for rowidx in range(df.shape[0] - 2, -1, -1):
        if df.iloc[rowidx, 0] == CycleEndValue:
            CycleBeginning = rowidx
            break
    df.drop(df.index[CycleBeginning - 1:], inplace = True)
    df.reset_index(drop=True, inplace = True)
    CycleEndValue = df.iloc[-1, 0]
    for rowidx in range(df.shape[0] - 2, -1, -1):
        if df.iloc[rowidx, 0] == CycleEndValue:
            CycleBeginning = rowidx
            break
    df.drop(df.index[0:CycleBeginning - 1], inplace = True)
    df.reset_index(drop=True, inplace = True)
    df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0])
    df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1])
    df.columns = ['Potential/V', 'Current/I']
    DataFrameListBeforeTreatment.append(df.copy(deep = True))
    df.columns = ['Potential/V vs RHE', 'Current/I']
    df.iloc[:, 0] += PotentialConstant
    df.iloc[:, 0] = df.iloc[:, 0].round(3)
    df.iloc[:, 1] = df.iloc[:, 1]

ForwardCapacitanceList = []
BackwardCapacitanceList = []
TotalChargeList = []
TotalCapacitanceList = []

for index, dataframe in enumerate(DataFrameList):
    forwardcapacitance = 0
    for rowidx in range(dataframe.shape[0]):
        if dataframe.iloc[rowidx, 0] >= End:
            forwardcapacitance += (dataframe.iloc[rowidx - 2, 0] - dataframe.iloc[rowidx - 1, 0]) * (dataframe.iloc[rowidx - 2, 1] + dataframe.iloc[rowidx - 1, 1])/ (2 * ScanRateList[index])
            if dataframe.iloc[rowidx, 0] >= Beginning:
                break
    ForwardCapacitanceList.append(forwardcapacitance)

for index, dataframe in enumerate(DataFrameList):
    backwardcapacitance = 0
    for rowidx in range(dataframe.shape[0] - 1, -1, -1):
        if dataframe.iloc[rowidx, 0] >= End:
            backwardcapacitance += (dataframe.iloc[rowidx - 2, 0] - dataframe.iloc[rowidx - 1, 0]) * (dataframe.iloc[rowidx - 2, 1] + dataframe.iloc[rowidx - 1, 1])/ (2 * ScanRateList[index])
            if dataframe.iloc[rowidx, 0] >= Beginning:
                break
    BackwardCapacitanceList.append(backwardcapacitance)

for idx in range(len(FileList)):
    TotalCapacitanceList.append(abs((ForwardCapacitanceList[idx] + BackwardCapacitanceList[idx]) / (2 * DeltaV)))
    TotalChargeList.append(abs(ForwardCapacitanceList[idx] + BackwardCapacitanceList[idx]))

TopCurveCalculation = []
TopLineCalculation = []
TopIntegralFinalValues = []

BottomCurveCalculation = []
BottomLineCalculation = []
BottomIntegralFinalValues = []

Top_MaximumCurrentValue = []
Top_RHEValueOfMax = []

Bottom_MaximumCurrentValue = []
Bottom_RHEValueOfMax = []

for idx in range(len(FileList)):
    TopCurveValue = 0
    TopLineValue = 0
    TopStartRowIndex = None
    TopEndRowIndex = None
    normaldf = DataFrameList[idx]
    modifieddf = normaldf.copy(deep = True)

    for rowidx in range(df.shape[0]):
        if (normaldf.iloc[rowidx, 0] == TopStartingIntegrationXValue):
            TopStartRowIndex = rowidx
            break
    for rowidx in range(df.shape[0]):
        if (normaldf.iloc[rowidx, 0] == TopEndingIntegrationXValue):
            TopEndRowIndex = rowidx
            break

    slope = (normaldf.iloc[TopEndRowIndex, 1] - normaldf.iloc[TopStartRowIndex, 1]) / (normaldf.iloc[TopEndRowIndex, 0] - normaldf.iloc[TopStartRowIndex, 0])
    yintercept = normaldf.iloc[TopEndRowIndex, 1] - (slope * normaldf.iloc[TopEndRowIndex, 0])
    for rowidx in range(TopStartRowIndex, TopEndRowIndex + 1):
        modifieddf.iloc[rowidx, 1] = (slope * normaldf.iloc[rowidx, 0]) + yintercept

    for rowidx in range(normaldf.shape[0]):
        if normaldf.iloc[rowidx, 0] >= TopStartingIntegrationXValue:
            TopCurveValue += (normaldf.iloc[rowidx - 2, 0] - normaldf.iloc[rowidx - 1, 0]) * (normaldf.iloc[rowidx - 2, 1] + normaldf.iloc[rowidx - 1, 1]) / (2 * ScanRateList[idx])
            if normaldf.iloc[rowidx, 0] >= TopEndingIntegrationXValue:
                break
    TopCurveCalculation.append(abs(TopCurveValue))

    for rowidx in range(modifieddf.shape[0]):
        if modifieddf.iloc[rowidx, 0] >= TopStartingIntegrationXValue:
            TopLineValue += (modifieddf.iloc[rowidx - 2, 0] - modifieddf.iloc[rowidx - 1, 0]) * (modifieddf.iloc[rowidx - 2, 1] + modifieddf.iloc[rowidx - 1, 1]) / (2 * ScanRateList[idx])
            if modifieddf.iloc[rowidx, 0] >= TopEndingIntegrationXValue:
                break
    TopLineCalculation.append(abs(TopLineValue))

    #Calculation for the top peak with the background subtracted
    Finding_Max_df = normaldf.loc[(normaldf['Potential/V vs RHE'] >= TopStartingIntegrationXValue) & (normaldf['Potential/V vs RHE'] <=TopEndingIntegrationXValue)]
    Value_Of_Max = Finding_Max_df['Current/I'].max()
    Index_Of_Max = Finding_Max_df['Current/I'].idxmax()
    NormalizedMaxValue = Value_Of_Max - modifieddf.iloc[Index_Of_Max, 1]
    Top_MaximumCurrentValue.append(NormalizedMaxValue)
    Top_RHEValueOfMax.append(Finding_Max_df['Potential/V vs RHE'].loc[Index_Of_Max])

for idx in range(len(FileList)):
    BottomCurveValue = 0
    BottomLineValue = 0
    BottomStartRowIndex = None
    BottomEndRowIndex = None
    regulardf = DataFrameList[idx]
    altereddf = regulardf.copy(deep=True)

    for rowidx in range(normaldf.shape[0] - 1, -1, -1):
        if regulardf.iloc[rowidx, 0] == BottomStartingIntegrationXValue:
            BottomStartRowIndex = rowidx
            break
    
    for rowidx in range(normaldf.shape[0] - 1, -1, -1):
        if regulardf.iloc[rowidx, 0] == BottomEndingIntegrationXValue:
            BottomEndRowIndex = rowidx
            break
    
    otherslope = (regulardf.iloc[BottomEndRowIndex, 1] - regulardf.iloc[BottomStartRowIndex, 1]) / (regulardf.iloc[BottomEndRowIndex, 0] - regulardf.iloc[BottomStartRowIndex, 0])
    otheryintercept = regulardf.iloc[BottomEndRowIndex, 1] - (otherslope * regulardf.iloc[BottomEndRowIndex, 0])

    for rowidx in range(BottomStartRowIndex, BottomEndRowIndex - 1, -1):
        altereddf.iloc[rowidx, 1] = (otherslope * regulardf.iloc[rowidx, 0]) + otheryintercept

    for rowidx in range(regulardf.shape[0] - 1, -1, -1):
        if regulardf.iloc[rowidx, 0] >= BottomStartingIntegrationXValue:
            BottomCurveValue += (regulardf.iloc[rowidx - 2, 0] - regulardf.iloc[rowidx - 1, 0]) * (regulardf.iloc[rowidx - 2, 1] + regulardf.iloc[rowidx - 1, 1]) / (2 * ScanRateList[idx])
            if regulardf.iloc[rowidx, 0] >= BottomEndingIntegrationXValue:
                
                break
    BottomCurveCalculation.append(abs(BottomCurveValue))

    for rowidx in range(altereddf.shape[0] - 1, -1, -1):
        if altereddf.iloc[rowidx, 0] >= BottomStartingIntegrationXValue:
            BottomLineValue += (altereddf.iloc[rowidx - 2, 0] - altereddf.iloc[rowidx - 1, 0]) * (altereddf.iloc[rowidx - 2, 1] + altereddf.iloc[rowidx - 1, 1]) / (2 * ScanRateList[idx])
            if altereddf.iloc[rowidx, 0] >= BottomEndingIntegrationXValue:
                break
    BottomLineCalculation.append(abs(BottomLineValue))

    #Calculation for the bottom peak with the background subtracted
    Finding_Min_df = regulardf.loc[(regulardf['Potential/V vs RHE'] >= BottomStartingIntegrationXValue) & (regulardf['Potential/V vs RHE'] <=BottomEndingIntegrationXValue)]
    Value_Of_Min = Finding_Min_df['Current/I'].min()
    Index_Of_Min = Finding_Min_df['Current/I'].idxmin()
    NormalizedMinValue = Value_Of_Min - altereddf.iloc[Index_Of_Min, 1]
    Bottom_MaximumCurrentValue.append(NormalizedMinValue)
    Bottom_RHEValueOfMax.append(Finding_Min_df['Potential/V vs RHE'].loc[Index_Of_Min])
    
for idx in range(len(FileList)):
    TopIntegralFinalValues.append(abs(TopCurveCalculation[idx] - TopLineCalculation[idx]))
    BottomIntegralFinalValues.append(abs(BottomCurveCalculation[idx] - BottomLineCalculation[idx]))

freecolumn = 0
for idx in range(len(FileList)):
    DataFrameListBeforeTreatment[idx].to_excel(writer, sheet_name = SheetName, index = False, startrow = freerow, startcol = freecolumn)
    freecolumn += len(DataFrameListBeforeTreatment[idx].columns)
    DataFrameList[idx].to_excel(writer, sheet_name = SheetName, index = False, startrow = freerow, startcol = freecolumn)
    freecolumn += len(DataFrameListBeforeTreatment[idx].columns) + 2

column = 0
for idx in range(len(FileList)):
    worksheet.write_string(freerow - 1, column, 'Segments 2-3')
    column += len(DataFrameListBeforeTreatment[idx].columns) + len(DataFrameListBeforeTreatment[idx].columns) + 2

chart = workbook.add_chart({'type': 'scatter', 'subtype': 'smooth'})
namecol = 0
categorycol = 2
valuescol = 3
for idx in range(len(FileList)):
    chart.add_series({
        'name': [SheetName, 0, namecol, 0, namecol],
        'categories': [SheetName, 2006, categorycol, 2006 + len(DataFrameList[idx]) - 1, categorycol],
        'values': [SheetName, 2006,  valuescol, 2006 + len(DataFrameList[idx]) - 1, valuescol],
    })
    namecol += 6
    categorycol += 6
    valuescol += 6
chart.set_title({'name': 'All CVs (Segments 2-3)'})
chart.set_x_axis({'min': 0, 'max': 1, 'name': 'Potential, V vs RHE'})
chart.set_y_axis({'name': 'Current, A'})
worksheet.insert_chart(16, freecolumn, chart)

TableHeader = []
for idx in range(len(FileList)):
    ForwardCapacitanceList[idx] = abs(ForwardCapacitanceList[idx])
    BackwardCapacitanceList[idx] = abs(BackwardCapacitanceList[idx])
    TableHeader.append(Legend[idx][34:])
TableList = [ScanRateList, ForwardCapacitanceList, BackwardCapacitanceList, TotalChargeList, TotalCapacitanceList, TopIntegralFinalValues, BottomIntegralFinalValues, Top_RHEValueOfMax, Top_MaximumCurrentValue, Bottom_RHEValueOfMax, Bottom_MaximumCurrentValue]
dftable = pd.DataFrame(TableList).T
dftable.columns = ['Scan Rate (V/s)', 'Double Layer Charge Pos (mA)', 'Double Layer Charge Neg (mA)', 'Total Charge (mA)', 'Capacitance (mF)', 'Top Charge (mCs)', 'Bottom Charge (mCs)', 'Epa', 'Ipa', 'Eca', 'Ica']
dftable.index = TableHeader
dftable.to_excel(writer, sheet_name = SheetName, startrow = 32 + len(FileList) + 2, startcol = freecolumn)
worksheet.write_string(31 + len(FileList) + 2, freecolumn, 'Segments 2-3  Table')

0

0

0

0

0

...

0

0

0

0

0

In [100]:
writer.save()